In [ ]:
%%writefile archs.py
import torch
import torch.nn as nn

# Arch1: Modified VGG-style convolutional layers for 28x28x1 input, n_params = 648,974
# Arch2: Modified ResNet model with BasicBlock (lesser blocks), n_params = 909,290.
# Arch3: Modified DenseNet model with Bottleneck and Transition layers, n_params = 100k
# Arch4: MobileNetV2-like model with DepthwiseSeparableConv layers, n_params = 149,566

class Arch1(nn.Module):
    def __init__(self, num_classes=62):
        super(Arch1, self).__init__()

        # Modified VGG-style convolutional layers for 28x28x1 input
        self.features = nn.Sequential(
            # Conv Layer 1 (Input: 28x28x1, Output: 28x28x64)
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Conv Layer 2 (Output: 28x28x64)
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 14x14x64

            # Conv Layer 3 (Output: 14x14x128)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Conv Layer 4 (Output: 14x14x128)
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 7x7x128
        )

        # Fully connected layer without hidden layers
        # Flatten the features from 7x7x128 to 6272 before feeding into the output layer
        self.classifier = nn.Linear(7 * 7 * 128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNetMod(nn.Module):
    def __init__(self, block, num_blocks, num_classes=62):
        super(ResNetMod, self).__init__()
        self.in_planes = 64

        # Adjust input conv layer for 28x28x1 input instead of 224x224x3
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # ResNet Layers
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        # The last two layers (residual blocks) are discarded as per your request.
        # So, we stop here, no layer3, layer4

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Adapted to small input size
        self.fc = nn.Linear(128 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the modified ResNet model
def Arch2(num_classes=62):
    return ResNetMod(BasicBlock, [2, 2], num_classes)

class Bottleneck(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(torch.relu(self.bn1(x)))
        out = self.conv2(torch.relu(self.bn2(out)))
        out = torch.cat([x, out], 1)
        return out

class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2)

    def forward(self, x):
        out = self.conv(torch.relu(self.bn(x)))
        out = self.pool(out)
        return out

class DenseNetMod(nn.Module):
    def __init__(self, num_classes=62, growth_rate=24, block_layers=[6, 6]):
        super(DenseNetMod, self).__init__()
        self.growth_rate = growth_rate
        num_planes = 2 * growth_rate  # Starting number of planes

        # Initial convolution layer
        self.conv1 = nn.Conv2d(1, num_planes, kernel_size=3, padding=1, bias=False)

        # Dense Block 1
        self.block1 = self._make_dense_layers(Bottleneck, num_planes, block_layers[0])
        num_planes += block_layers[0] * growth_rate
        self.trans1 = Transition(num_planes, num_planes // 2)
        num_planes = num_planes // 2

        # Dense Block 2
        self.block2 = self._make_dense_layers(Bottleneck, num_planes, block_layers[1])
        num_planes += block_layers[1] * growth_rate
        self.trans2 = Transition(num_planes, num_planes // 2)
        num_planes = num_planes // 2

        # Global average pooling and fully connected layer
        self.bn = nn.BatchNorm2d(num_planes)
        self.fc = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_channels, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_channels, self.growth_rate))
            in_channels += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.trans1(out)
        out = self.block2(out)
        out = self.trans2(out)
        out = torch.relu(self.bn(out))
        out = torch.nn.functional.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the modified ResNet model
def Arch3(num_classes=62):
    return DenseNetMod(num_classes=num_classes, growth_rate=12, block_layers=[6, 6])

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = torch.relu(self.bn1(self.depthwise(x)))
        out = self.bn2(self.pointwise(out))
        return out

class Arch4(nn.Module):
    def __init__(self, num_classes=62):
        super(Arch4, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.dw_conv1 = DepthwiseSeparableConv(32, 64)
        self.dw_conv2 = DepthwiseSeparableConv(64, 128, stride=2)
        self.dw_conv3 = DepthwiseSeparableConv(128, 128)
        self.dw_conv4 = DepthwiseSeparableConv(128, 256, stride=2)
        self.dw_conv5 = DepthwiseSeparableConv(256, 256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.dw_conv1(out)
        out = self.dw_conv2(out)
        out = self.dw_conv3(out)
        out = self.dw_conv4(out)
        out = self.dw_conv5(out)
        out = self.global_avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

if __name__ == "__main__":

    input_image = torch.rand(1, 1, 28, 28)
    model = Arch1(num_classes=62)
    output = model(input_image)
    print(output.shape)
    model = Arch2(num_classes=62)
    output = model(input_image)
    print(output.shape)
    model = Arch3(num_classes=62)
    output = model(input_image)
    print(output.shape)
    model = Arch4(num_classes=62)
    output = model(input_image)
    print(output.shape)

Writing archs.py


In [ ]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNetMod(nn.Module):
    def __init__(self, block, num_blocks, num_classes=62):
        super(ResNetMod, self).__init__()
        self.in_planes = 16  # Reduced input channels

        # Adjust input conv layer for 28x28x1 input instead of 224x224x3
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        # ResNet Layers with fewer blocks and planes
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)

        # Average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Adapted to small input size
        self.fc = nn.Linear(32 * block.expansion, num_classes)  # Adjusted output size

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the modified ResNet model with reduced parameters
def Arch2(num_classes=62):
    return ResNetMod(BasicBlock, [1, 1], num_classes)  # Reduced blocks for further parameter reduction

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import EMNIST
from torch.utils.data import random_split
from sklearn.metrics import precision_score, f1_score
import torch.nn.functional as F
# from archs import *

class ModifiedCrossEntropyLoss(nn.Module):
    def __init__(self, penalty_weight=0.1):
        super(ModifiedCrossEntropyLoss, self).__init__()
        self.penalty_weight = penalty_weight

    def forward(self, inputs, targets):
        # Calculate probabilities using softmax
        probs = F.softmax(inputs, dim=1)  # Get probabilities from raw logits

        # Standard cross-entropy loss for the true class
        loss_ce = torch.log(probs[range(targets.size(0)), targets] + 1e-12).mean()

        # Calculate the penalty for all classes except the true class
        penalty = self.penalty_weight * (torch.sum(torch.log(1 - probs + 1e-12), dim=1) -
                                          torch.log(1 - probs[range(targets.size(0)), targets] + 1e-12))

        # Final loss
        total_loss = loss_ce + penalty.mean()
        return -total_loss

class ImageClassifier:
    def __init__(self, network, optimizer, criterion, l2_lambda=0.01):
        self.network = network
        self.optimizer = optimizer
        self.criterion = criterion
        self.l2_lambda = l2_lambda
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.network.to(self.device)

    def _regularize(self, network, l2_lambda):
        # Compute L2 regularization
        l2_reg = 0.0
        for param in network.parameters():
            l2_reg += torch.norm(param, 2)
        return l2_lambda * l2_reg

    def compute_loss(self, outputs, targets, l2_lambda=0.01, regularize = False):
        # Compute the cross-entropy loss
        ce_loss = self.criterion(outputs, targets)

        if regularize:
            # Compute regularization loss
            l2_reg = self._regularize(self.network, l2_lambda)

            return ce_loss + l2_reg

        return ce_loss

    def compute_metrics(self, preds, targets):
        """Helper function to compute accuracy, precision, and F1 score."""
        # Ensure preds are already in label form (if not already converted)
        if preds.dim() > 1:  # Check if preds need reduction
            preds = preds.argmax(dim=1)  # Get the predicted labels

        preds = preds.cpu().numpy()  # Convert predictions to NumPy
        targets = targets.cpu().numpy()  # Convert true labels to NumPy

        # Compute accuracy
        accuracy = (preds == targets).mean()

        # Compute precision and F1 score using scikit-learn
        precision = precision_score(targets, preds, average='weighted', zero_division=0)
        f1 = f1_score(targets, preds, average='weighted')

        return accuracy, precision, f1

    def train(self, train_loader, val_loader, n_epochs=10, patience=3):
        best_val_loss = float('inf')
        current_patience = 0

        for epoch in range(n_epochs):
            # Train
            self.network.train()
            train_loss = 0.0
            all_preds = []
            all_targets = []

            # Use tqdm for progress bar and set dynamic description
            train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Training Epoch {epoch + 1}')
            for batch_idx, (data, target) in train_bar:
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with loss and accuracy
                current_accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                train_bar.set_postfix(loss=train_loss / (batch_idx + 1), accuracy=current_accuracy)

            # Calculate final metrics for training
            all_preds = torch.cat(all_preds)
            all_targets = torch.cat(all_targets)
            train_accuracy, train_precision, train_f1 = self.compute_metrics(all_preds, all_targets)

            # Validate
            self.network.eval()
            val_loss = 0.0
            val_preds = []
            val_targets = []

            # Use tqdm for validation progress bar
            val_bar = tqdm(val_loader, desc='Validating')
            with torch.no_grad():
                for data, target in val_bar:
                    data, target = data.to(self.device), target.to(self.device)

                    # Forward pass
                    outputs = self.network(data)

                    # Compute loss
                    loss = self.compute_loss(outputs, target)
                    val_loss += loss.item()

                    # Gather predictions and true labels
                    preds = outputs.argmax(dim=1)
                    val_preds.append(preds)
                    val_targets.append(target)

                    # Update progress bar with validation loss and accuracy
                    val_accuracy, _, _ = self.compute_metrics(torch.cat(val_preds), torch.cat(val_targets))
                    val_bar.set_postfix(val_loss=val_loss / len(val_loader), accuracy=val_accuracy)

            # Calculate final validation metrics
            val_preds = torch.cat(val_preds)
            val_targets = torch.cat(val_targets)
            val_accuracy, val_precision, val_f1 = self.compute_metrics(val_preds, val_targets)

            # Print epoch statistics
            train_loss /= len(train_loader)
            val_loss /= len(val_loader)
            print(f'Epoch {epoch + 1}/{n_epochs}, '
                  f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
                  f'Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}, '
                  f'Train Prec: {train_precision:.4f}, Val Prec: {val_precision:.4f}, '
                  f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                current_patience = 0
            else:
                current_patience += 1
                if current_patience >= patience:
                    print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
                    break

    def test(self, test_loader):
        self.network.eval()
        test_loss = 0.0
        correct = 0
        all_preds = []
        all_targets = []

        # Use tqdm for test progress bar
        test_bar = tqdm(test_loader, desc='Testing')
        with torch.no_grad():
            for data, target in test_bar:
                data, target = data.to(self.device), target.to(self.device)

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                test_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with test loss and accuracy
                accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                test_bar.set_postfix(loss=test_loss / len(test_loader), accuracy=accuracy)

        # Calculate final test metrics
        all_preds = torch.cat(all_preds)
        all_targets = torch.cat(all_targets)
        accuracy, precision, f1 = self.compute_metrics(all_preds, all_targets)

        test_loss /= len(test_loader)
        print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, F1 Score: {f1:.2f}')

# Define transformation for the images
transform = transforms.Compose([
    transforms.ToTensor(),            # Convert to tensor (1 channel)
    transforms.Normalize((0.5), (0.5))  # Normalize for RGB
])

# Download the EMNIST ByClass dataset
emnist_dataset = EMNIST(root='data', split='byclass', train=True, download=True, transform=transform)
test_dataset = EMNIST(root='data', split='byclass', train=False, download=True, transform=transform)

# Define the sizes for the training and validation sets
train_size = int(0.85 * len(emnist_dataset))  # 80% for training
val_size = len(emnist_dataset) - train_size   # remaining 20% for validation

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(emnist_dataset, [train_size, val_size])

print(f'Training set size: {len(train_dataset)}')
print(f'Validation set size: {len(val_dataset)}')
print(f'Test set size: {len(test_dataset)}')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024)
test_loader = DataLoader(test_dataset, batch_size=1024)

# Initialize the neural network, optimizer, and criterion
model = Arch2(num_classes=62)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = ModifiedCrossEntropyLoss(penalty_weight=0.1)

# Create an instance of ImageClassifier
classifier = ImageClassifier(model, optimizer, criterion)

# Train the classifier
classifier.train(train_loader, val_loader, n_epochs=10, patience=3)

# Test the classifier
classifier.test(test_loader)

torch.save(model.state_dict(), 'model2.pth')

Training set size: 593242
Validation set size: 104690
Test set size: 116323


Validating: 100%|██████████| 103/103 [00:25<00:00,  4.01it/s, accuracy=0.694, val_loss=1.28]


Epoch 1/10, Train Loss: 2.1559, Val Loss: 1.2816, Train Acc: 0.5149, Val Acc: 0.6942, Train Prec: 0.5002, Val Prec: 0.6456, Train F1: 0.4496, Val F1: 0.6378


Validating: 100%|██████████| 103/103 [00:25<00:00,  4.00it/s, accuracy=0.748, val_loss=0.913]


Epoch 2/10, Train Loss: 0.9153, Val Loss: 0.9133, Train Acc: 0.7620, Val Acc: 0.7478, Train Prec: 0.7303, Val Prec: 0.7300, Train F1: 0.7235, Val F1: 0.7141


...

Epoch 9/10, Train Loss: 0.4923, Val Loss: 0.5373, Train Acc: 0.8450, Val Acc: 0.8289, Train Prec: 0.8267, Val Prec: 0.8124, Train F1: 0.8271, Val F1: 0.8104


Validating: 100%|██████████| 103/103 [00:25<00:00,  4.05it/s, accuracy=0.843, val_loss=0.5]


Epoch 10/10, Train Loss: 0.4844, Val Loss: 0.4996, Train Acc: 0.8467, Val Acc: 0.8429, Train Prec: 0.8289, Val Prec: 0.8314, Train F1: 0.8293, Val F1: 0.8282


Testing: 100%|██████████| 114/114 [00:28<00:00,  4.00it/s, accuracy=0.843, loss=0.499]

Test Loss: 0.4986, Accuracy: 0.84%, Precision: 0.83, F1 Score: 0.83


In [ ]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNetMod(nn.Module):
    def __init__(self, block, num_blocks, num_classes=62):
        super(ResNetMod, self).__init__()
        self.in_planes = 32  # Reduced from 64 to 32

        # Adjust input conv layer for 28x28x1 input instead of 224x224x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False)  # Reduced filters
        self.bn1 = nn.BatchNorm2d(32)

        # ResNet Layers with reduced planes
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)  # Reduced from 64 to 32 filters
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)  # Reduced from 128 to 64 filters

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Adapted to small input size
        self.fc = nn.Linear(64 * block.expansion, num_classes)  # Reduced input size

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the modified ResNet model
def Arch2_2(num_classes=62):
    return ResNetMod(BasicBlock, [2, 2], num_classes)

import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import EMNIST
from torch.utils.data import random_split
from sklearn.metrics import precision_score, f1_score
import torch.nn.functional as F
# from archs import *

class ModifiedCrossEntropyLoss(nn.Module):
    def __init__(self, penalty_weight=0.1):
        super(ModifiedCrossEntropyLoss, self).__init__()
        self.penalty_weight = penalty_weight

    def forward(self, inputs, targets):
        # Calculate probabilities using softmax
        probs = F.softmax(inputs, dim=1)  # Get probabilities from raw logits

        # Standard cross-entropy loss for the true class
        loss_ce = torch.log(probs[range(targets.size(0)), targets] + 1e-12).mean()

        # Calculate the penalty for all classes except the true class
        penalty = self.penalty_weight * (torch.sum(torch.log(1 - probs + 1e-12), dim=1) -
                                          torch.log(1 - probs[range(targets.size(0)), targets] + 1e-12))

        # Final loss
        total_loss = loss_ce + penalty.mean()
        return -total_loss

class ImageClassifier:
    def __init__(self, network, optimizer, criterion, l2_lambda=0.01):
        self.network = network
        self.optimizer = optimizer
        self.criterion = criterion
        self.l2_lambda = l2_lambda
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.network.to(self.device)

    def _regularize(self, network, l2_lambda):
        # Compute L2 regularization
        l2_reg = 0.0
        for param in network.parameters():
            l2_reg += torch.norm(param, 2)
        return l2_lambda * l2_reg

    def compute_loss(self, outputs, targets, l2_lambda=0.01, regularize = False):
        # Compute the cross-entropy loss
        ce_loss = self.criterion(outputs, targets)

        if regularize:
            # Compute regularization loss
            l2_reg = self._regularize(self.network, l2_lambda)

            return ce_loss + l2_reg

        return ce_loss

    def compute_metrics(self, preds, targets):
        """Helper function to compute accuracy, precision, and F1 score."""
        # Ensure preds are already in label form (if not already converted)
        if preds.dim() > 1:  # Check if preds need reduction
            preds = preds.argmax(dim=1)  # Get the predicted labels

        preds = preds.cpu().numpy()  # Convert predictions to NumPy
        targets = targets.cpu().numpy()  # Convert true labels to NumPy

        # Compute accuracy
        accuracy = (preds == targets).mean()

        # Compute precision and F1 score using scikit-learn
        precision = precision_score(targets, preds, average='weighted', zero_division=0)
        f1 = f1_score(targets, preds, average='weighted')

        return accuracy, precision, f1

    def train(self, train_loader, val_loader, n_epochs=10, patience=3):
        best_val_loss = float('inf')
        current_patience = 0

        for epoch in range(n_epochs):
            # Train
            self.network.train()
            train_loss = 0.0
            all_preds = []
            all_targets = []

            # Use tqdm for progress bar and set dynamic description
            train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Training Epoch {epoch + 1}')
            for batch_idx, (data, target) in train_bar:
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with loss and accuracy
                current_accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                train_bar.set_postfix(loss=train_loss / (batch_idx + 1), accuracy=current_accuracy)

            # Calculate final metrics for training
            all_preds = torch.cat(all_preds)
            all_targets = torch.cat(all_targets)
            train_accuracy, train_precision, train_f1 = self.compute_metrics(all_preds, all_targets)

            # Validate
            self.network.eval()
            val_loss = 0.0
            val_preds = []
            val_targets = []

            # Use tqdm for validation progress bar
            val_bar = tqdm(val_loader, desc='Validating')
            with torch.no_grad():
                for data, target in val_bar:
                    data, target = data.to(self.device), target.to(self.device)

                    # Forward pass
                    outputs = self.network(data)

                    # Compute loss
                    loss = self.compute_loss(outputs, target)
                    val_loss += loss.item()

                    # Gather predictions and true labels
                    preds = outputs.argmax(dim=1)
                    val_preds.append(preds)
                    val_targets.append(target)

                    # Update progress bar with validation loss and accuracy
                    val_accuracy, _, _ = self.compute_metrics(torch.cat(val_preds), torch.cat(val_targets))
                    val_bar.set_postfix(val_loss=val_loss / len(val_loader), accuracy=val_accuracy)

            # Calculate final validation metrics
            val_preds = torch.cat(val_preds)
            val_targets = torch.cat(val_targets)
            val_accuracy, val_precision, val_f1 = self.compute_metrics(val_preds, val_targets)

            # Print epoch statistics
            train_loss /= len(train_loader)
            val_loss /= len(val_loader)
            print(f'Epoch {epoch + 1}/{n_epochs}, '
                  f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
                  f'Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}, '
                  f'Train Prec: {train_precision:.4f}, Val Prec: {val_precision:.4f}, '
                  f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                current_patience = 0
            else:
                current_patience += 1
                if current_patience >= patience:
                    print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
                    break

    def test(self, test_loader):
        self.network.eval()
        test_loss = 0.0
        correct = 0
        all_preds = []
        all_targets = []

        # Use tqdm for test progress bar
        test_bar = tqdm(test_loader, desc='Testing')
        with torch.no_grad():
            for data, target in test_bar:
                data, target = data.to(self.device), target.to(self.device)

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                test_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with test loss and accuracy
                accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                test_bar.set_postfix(loss=test_loss / len(test_loader), accuracy=accuracy)

        # Calculate final test metrics
        all_preds = torch.cat(all_preds)
        all_targets = torch.cat(all_targets)
        accuracy, precision, f1 = self.compute_metrics(all_preds, all_targets)

        test_loss /= len(test_loader)
        print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, F1 Score: {f1:.2f}')

# Define transformation for the images
transform = transforms.Compose([
    transforms.ToTensor(),            # Convert to tensor (1 channel)
    transforms.Normalize((0.5), (0.5))  # Normalize for RGB
])

# Download the EMNIST ByClass dataset
emnist_dataset = EMNIST(root='data', split='byclass', train=True, download=True, transform=transform)
test_dataset = EMNIST(root='data', split='byclass', train=False, download=True, transform=transform)

# Define the sizes for the training and validation sets
train_size = int(0.85 * len(emnist_dataset))  # 80% for training
val_size = len(emnist_dataset) - train_size   # remaining 20% for validation

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(emnist_dataset, [train_size, val_size])

print(f'Training set size: {len(train_dataset)}')
print(f'Validation set size: {len(val_dataset)}')
print(f'Test set size: {len(test_dataset)}')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024)
test_loader = DataLoader(test_dataset, batch_size=1024)

# Initialize the neural network, optimizer, and criterion
model = Arch2_2(num_classes=62)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = ModifiedCrossEntropyLoss(penalty_weight=0.1)

# Create an instance of ImageClassifier
classifier = ImageClassifier(model, optimizer, criterion)

# Train the classifier
classifier.train(train_loader, val_loader, n_epochs=10, patience=3)

# Test the classifier
classifier.test(test_loader)

torch.save(model.state_dict(), 'model2_2.pth')

Training set size: 593242
Validation set size: 104690
Test set size: 116323


Validating: 100%|██████████| 103/103 [00:29<00:00,  3.48it/s, accuracy=0.837, val_loss=0.532]


Epoch 1/10, Train Loss: 0.9716, Val Loss: 0.5317, Train Acc: 0.7563, Val Acc: 0.8366, Train Prec: 0.7434, Val Prec: 0.8241, Train F1: 0.7282, Val F1: 0.8133


Validating: 100%|██████████| 103/103 [00:29<00:00,  3.51it/s, accuracy=0.848, val_loss=0.453]


Epoch 2/10, Train Loss: 0.4529, Val Loss: 0.4531, Train Acc: 0.8544, Val Acc: 0.8480, Train Prec: 0.8360, Val Prec: 0.8382, Train F1: 0.8358, Val F1: 0.8308


...

Epoch 9/10, Train Loss: 0.3671, Val Loss: 0.3812, Train Acc: 0.8737, Val Acc: 0.8702, Train Prec: 0.8658, Val Prec: 0.8611, Train F1: 0.8605, Val F1: 0.8584


Validating: 100%|██████████| 103/103 [00:29<00:00,  3.54it/s, accuracy=0.87, val_loss=0.378]


Epoch 10/10, Train Loss: 0.3631, Val Loss: 0.3776, Train Acc: 0.8748, Val Acc: 0.8703, Train Prec: 0.8655, Val Prec: 0.8629, Train F1: 0.8618, Val F1: 0.8556


Testing: 100%|██████████| 114/114 [00:33<00:00,  3.45it/s, accuracy=0.87, loss=0.38]

Test Loss: 0.3798, Accuracy: 0.87%, Precision: 0.87, F1 Score: 0.86
